# In class exercise: imports, kmers, and *de_bruijn* graphs

In [ ]:
import random
import toyplot
LAYOUT = toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges())


### kmers and the debruijn graph

kmers are substrings of length k of a larger string. The de Bruijn graph is a mathematical construct used in genome assembly and other network based problems. It relies on the conjecture that a sequence can be reconstructed by substrings of that sequence if all substrings are present and they overlap by n-1. One example type of substring would be a genomic short read sequence. However, these may not overlap perfectly by n-1. But, we can use smaller kmers from those reads to get more substrings that may overlap by n-1 and thus be used to reconstruct the sequence. See image for example. 



![image.png](https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/K-mer-example.png/440px-K-mer-example.png)
https://en.wikipedia.org/wiki/K-mer

### Genome/contig assembly
In this notebook we are going to write functions to assemble a contig from a set of reads broken into kmers. 

### (1) Write a function to generate a random sequence of dna


In [ ]:
def random_sequence(nbases):
    return 

### (2) Write a function to return all kmers for a string
Return the kmers as a count dictionary with the number of times each kmer was observed. 

In [ ]:
def get_kmers(target):
    return 

### (3) Test our function on a random sequence

In [ ]:
target = random_sequence(50)
kmers = get_kmers(target, 3)
kmers

### (4) Write a function to generate sequence reads of len N
This will take arguments `nreads` and `readlength`. 

In [ ]:
def get_random_reads(nreads, readlength):
    return 

### (5) A function to break reads into kmers

In [ ]:
def reads_to_kmers(reads, k):
    "stores kmers to dict uses .update() to join together kmer dict keys"
    kmers = {}
    #...
    return kmers

### (6) Test our functions

In [ ]:
## params 
tlen = 100
nreads = 200
rlen = 25
k = 8

## call funcs
random.seed(123)
target = random_sequence(tlen)
reads = get_random_reads(target, nreads, rlen)
kmers = reads_to_kmers(reads, k)

In [ ]:
kmers.keys()

### (7) Write a function to return a debruijn graph 
This should return a list of tuples where each tuple is a (kmer, kmer) pair where the two kmers overlap identically over n-1 of their length. This is the definition of a [deBruijn graph](https://en.wikipedia.org/wiki/De_Bruijn_graph). Use two nested for loops to compare all kmers to each other. Use slicing to compare the `[1:]` index to the `[:-1]` index of the other to test for `n-1` overlap.  

In [ ]:
def get_debruijn_edges(kmers):
    edges = set()
    # ...
    return edges

In [ ]:
edges = get_debruijn_edges(kmers)
edges

### (8) Plot the deBruijn graph
Use the toyplot function below to plot the deBruijn graph generated with the following code. 

In [ ]:
def plot_debruijn_graph(edges):
    "returns a toyplot graph of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        tmarker=">", 
        width=600,
        vlstyle={"font-size": "8px"},
        layout=LAYOUT)
    return graph

In [ ]:
## plot as directed graph
plot_debruijn_graph(edges);

### (9) Test for a eulerian path
When there are many repeats in a sequence then there may be multiple paths through the graph that touch each edge once. Or, if the graph is not complete, for example if there are too few kmers to complete the graph, then a full eulerian path between all kmers cannot be found. The function to find the eulerian path is a bit complicated so for now we will just import a working 


In [ ]:
from eulerian import eulerian_path

## this will raise an error if the path does not exist
path = eulerian_path(edges)

### Exporting function to .py files
Follow the lecture instructions to now copy all of the functions we defined above into a new text file which you can create from your jupyter dashboard by selecting `[new]/[text file]`.
We will then try using these functions again imported from our new python file. It is important that you name the file `debruijn.py`. 

### Imports: testing on a simple example 

In [ ]:
import debruijn

In [ ]:
target = "AAABBBBA"
kmers = debruijn.get_kmers(target, 3)
kmers

In [ ]:
edges = debruijn.get_debruijn_edges(kmers)
edges

In [ ]:
plot_debruijn_graph(edges);

### Repeats can create ambiguity in the de Bruijn graph
The de Bruijn graph represents a path to contructing the full genome by walking the path along directed edges to each node. This includes cylic walks along repeated elements (e.g., AA to AA) although such moves do not appear super clearly in plots we've generated. Repeat elements are particularly troubling for de bruijn graphs because they create ambiguity where there can be more than one way to walk across all edges of the graph. Let's test assembling a large sequence by decomposing kmers of different size from a different depth of 50 bp reads. 

In [ ]:
import debruijn
import eulerian
import random

random.seed(123)
target = debruijn.random_sequence(500)

In [ ]:
## a dictionary to store our results
results = {}

## iterate over kmer sizes
for kmersize in [10, 30, 100]:
    for nreads in [50, 100, 500, 1000]:
        
        ## store zero starting value
        name = (kmersize, nreads)
        results[name] = 0
        
        ## test over multiple replicates
        for replicate in range(10):
            
            ## call funcs
            reads = debruijn.get_reads(target, nreads=nreads, rlen=50)
            kmers = debruijn.reads_to_kmers(reads, kmersize)
            edges = debruijn.get_debruijn_edges(kmers)
            
            ## test for eulerian walk
            try:
                path = eulerian.eulerian_path(edges)
                results[name] += 1
                   
            except Exception:
                pass
                        

In [ ]:
## show results
results

### How does kmer size affect the results?

https://en.wikipedia.org/wiki/K-mer